In [2]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [3]:
import os
os.chdir("../../")

In [4]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [5]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
sbert_embeddings.shape

torch.Size([5568, 384])

# Baseline: FFN using signatures

In [8]:
dim_reduce_methods = ["umap"]
num_epochs = 100
batch=64
log_sig = True
log_signature_dimensions_and_sig_depths = [ (10, 3)]#[(28, 2), (10, 3), (6, 4)]
hidden_dim_sizes = [[32,32]]#[[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-3]#[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
patience=4
validation_metric = "f1"

In [9]:
ffn_logsignature_umap, best_ffn_logsignature_umap, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    log_signature=log_sig,
    dim_reduce_methods=dim_reduce_methods,
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths, 
    path_indices=None,
    data_split_seed=123,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the path signature of the history for each item in the dataframe
























































































































100%|██████████| 5568/5568 [00:12<00:00, 462.23it/s]









100%|██████████| 1/1 [00:33<00:00, 33.30s/it]


100%|██████████| 1/1 [00:33<00:00, 33.31s/it]

100%|██████████| 1/1 [01:59<00:00, 119.74s/it]


[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the path signature of the history for each item in the dataframe


100%|██████████| 5568/5568 [00:11<00:00, 467.68it/s]


In [10]:
ffn_logsignature_umap

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,...,seed,gamma,k_fold,batch_size,model_id,input_dim,dimension,sig_depth,method,log_signature
0,focal,0.717826,0.711019,"[0.7553719008264462, 0.6666666666666666]",0.388626,0.615658,0.535053,"[0.728643216080402, 0.34146341463414637]","[32, 32]",0.2,...,0,2,False,64,0.0,769,10,3,umap,True
0,focal,0.670162,0.657849,"[0.7227564102564101, 0.5929411764705882]",0.285560,0.672598,0.574355,"[0.7788461538461537, 0.3698630136986301]","[32, 32]",0.2,...,1,2,False,64,0.0,769,10,3,umap,True
0,focal,0.654909,0.629043,"[0.726998491704374, 0.5310880829015544]",0.242103,0.604982,0.530376,"[0.717557251908397, 0.3431952662721893]","[32, 32]",0.2,...,12,2,False,64,0.0,769,10,3,umap,True
0,focal,0.637750,0.625861,"[0.6925566343042071, 0.5591647331786543]",0.285084,0.594306,0.519298,"[0.7091836734693876, 0.32941176470588235]","[32, 32]",0.2,...,123,2,False,64,0.0,769,10,3,umap,True
0,focal,0.684461,0.678401,"[0.7225481978206202, 0.6342541436464089]",0.292300,0.640569,0.560400,"[0.7481296758104737, 0.37267080745341613]","[32, 32]",0.2,...,1234,2,False,64,0.0,769,10,3,umap,True


In [11]:
best_ffn_logsignature_umap["f1"].mean()

0.6604346437775931

In [12]:
np.stack(best_ffn_logsignature_umap["f1_scores"]).mean(axis=0)

array([0.72404633, 0.59682296])